In [1]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pickle
from collections import Counter
from collections import namedtuple
import re
import xlwings as xw
import pandas as pd
import numpy as np

In [2]:
path_to_names = "D:/"
path_to_patronymics = "D:/"

In [3]:
def distance(s, t):
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [4]:
def Brute(tmp):
    res = []
    for t in tmp:
        name=""
        surname=""
        patronymic=""
        words = t.split(" ")
        for w in words:
#             print ("defining " + w)
            w = w.lower()
            w.strip()
            if (name==""):
                for n in all_names:
                    if w==n:
#                         print("found a name " + n)
                        name=n
                        break
            
            if (patronymic==""):
                for p in all_patronymics:
                    if w==p:
#                         print("found a patronymic " + p)
                        patronymic=p
                        break
                        
            if (name!=w and patronymic!=w and surname==""):
                surname=w
        
        if (surname==""):
            surname="Not found"
        if (name==""):
            name="Not found"
        if (patronymic==""):
            patronymic="Not found"
        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname +" "+ name +" "+ patronymic)
    return res

In [5]:
def SimpleNotBruteEnough(tmp):
    res = []

    for t in tmp:
        name=""
        surname=""
        patronymic=""
        t = t.lower()
        words = t.split(" ")
        
        for w in words:
            w.strip()
            if (name==""):
                for n in all_names:
                    if w==n:
                        name=n
                        break
            
            if (patronymic==""):
                for p in all_patronymics:
                    if w==p:
                        patronymic=p
                        break
                        
            if (name!=w and patronymic!=w and surname==""):
                surname=w
        
        if(name in words): words.remove(name)        
        if(patronymic in words): words.remove(patronymic)  
        if(surname in words): words.remove(surname)  
        
        if (surname==""):
            surname="Not found"
        
        if (name=="")&(len(words)>0):
            foundWord = ""
            for w in words:
                w.strip()
                minDist = len(w)
                for n in all_names:
                    dist = distance(w, n)
                    if dist<minDist:
                        minDist=dist
                        name=n.title() + " (" + str(round((1 - minDist/len(w))*100)) + "% " + w+ ")"
                        foundWord = w
            words.remove(foundWord)
        
        if (patronymic=="")&(len(words)>0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = distance(w, p)
                    if dist<minDist:
                        minDist=dist
                        patronymic=p.title() + " (" + str(round((1 - minDist/len(w))*100)) + "% " + w+ ")"
                        foundWord = w
            words.remove(foundWord)

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname +" "+ name +" "+ patronymic)
        
    return res

In [6]:
with open(path_to_names+"NamesFromGufo.pickle","rb") as f:
    tmpNames=pickle.load(f)
with open(path_to_patronymics+"PatronymicsFromGufo.pickle","rb") as f:
    tmpPatronymics=pickle.load(f)

In [7]:
all_names=[]
all_patronymics=[]

In [8]:
for i in tmpNames:
    all_names.append(i)
for i in tmpPatronymics:
    all_patronymics.append(i)

In [9]:
with open("D:/test_changed.pickle","rb") as f:
    tmp=pickle.load(f)

In [10]:
#testing_res=Brute(tmp)
testing_res=SimpleNotBruteEnough(tmp)
for fio in testing_res:
    print(fio)

Стрелкова Глафира Глебовна
Лыткин Константин Николаевич
Субботина Оксана Георгиевна (90% Георгьевна)
Бурова Милица Евгеньевна
Мэлор Афросин (57% Сорокин) Арсеньевич
Зуева Лариса Лаврентьевна
Кузьмин Авдей Матвеевич
Дементьева Юлия Леонидовна
Сорокин Демьян Егорович
Попова Екатерина Михаиловна
Моисеева Варвара Руслановна
Турова Антонина Кондратовна
Игнатова Алевтина Глебовна (88% Улебовна)
Волков Фрол Артёмович
Селезнёва Галина Якуновна
Нестеров Василий Валерьевич
Сидоров Виктор Михаилович
Мясникова Олимпиада Андреевна
Овчинников Антонин Протасьевич
Путин Богдан Христофорович
Прохорова Дарья Авксентьевна
Суворова Галина Антониновна
Мэлс Пров (60% Попов) Арсеньевич
Самсонов Ростислав Антонович
Семёнова Тамара Варламовна
Васильева Ангелина Святославовна
Егорова Оксана Григорьевна
Фокина Лидия Олеговна
Зайцев Кондрат Валерьянович
Никонов Ириней Петрович (88% Пётрович)
Мамонтова Татьяна Федосеевна
Корнилов Протасий Павлович
Колобов Семён Станиславович
Комаров Матвей Олегович
Поляков Протаси

In [11]:
errs=0
for i in testing_res:
    if (i.find("Not Found")!=-1)|(i.find("%")!=-1):
        print (i)
        errs+=1
print("Всего не уверены в: " + str(errs))

Субботина Оксана Георгиевна (90% Георгьевна)
Мэлор Афросин (57% Сорокин) Арсеньевич
Игнатова Алевтина Глебовна (88% Улебовна)
Мэлс Пров (60% Попов) Арсеньевич
Никонов Ириней Петрович (88% Пётрович)
Рыбаков Глеб (75% Улеб) Борисович
Анисимов Амур (60% Дамир) Еремеевич
Дамирович Альберт Гаспарович (38% Беспалов)
Улеб Аронос (43% Морозов) Язонович (75% Яковович)
Наина Дина (80% Юдина) Степановна
Мэлс Агав (40% Рогов) Валерьевич
Улебович Борис Горгич (43% Горшков)
Пётровна Вероника Евдокимовна (90% Евдокимова)
Германн Увар (67% Уваров) Авдеевич
Лора Меркул (56% Меркушева) Романовна
Федотова Светлана Никиевна (89% Никитевна)
Беспалов Аркадий Германович (91% Германнович)
Никитевич Ростислав Вельмирович (44% Мельников)
Дамировна Наталья Лазаревна (88% Лазарева)
Пётрович Агафон Исевич (43% Киселёв)
Кононов Валентин Дмитриевич (90% Дмитрьевич)
Лукьевич Игорь Фомич (80% Фомин)
Ильяовна Татьяна Кононовна (88% Кононова)
Медведев Иван Максович (75% Мэлсович)
Воронцова Марфа Георгиевна (90% Георгьев